Task1_0724. Wine 데이터셋에 대하여 SVM 모델에 3개의 커널을 적용하여 학습 및 평가 결과를 출력하세요.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

wine = datasets.load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

kernels = ['linear', 'poly', 'rbf']

for kernel in kernels:
  svm = SVC(kernel=kernel, random_state=42)
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)

  print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.97
Accuracy: 0.97
Accuracy: 1.00


Task2_0724. breast_cancer dataset으로 랜덤포레스트를 적용하여 모델링 및 평가를 아래의 하이퍼 파라미터를 이용하여 수행한 후 최적의 하이퍼파라미터를 구하세요.

- 'n_estimators': [50, 100, 200],
- 'max_depth': [None, 10, 20],
- 'max_features': ['auto', 'sqrt', 'log2'],
- 'min_samples_split': [2, 5, 10],
- 'min_samples_leaf': [1, 2, 4]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#clf = RandomForestClassifier(n_estimators=100, random_state=42) # 9649
#clf = RandomForestClassifier(n_estimators=200, random_state=42) # 9649
#clf = RandomForestClassifier(n_estimators=50, random_state=42) # 9649

#clf = RandomForestClassifier(max_depth=None, random_state=42) # 9649
#clf = RandomForestClassifier(max_depth=10, random_state=42) # 9649
#clf = RandomForestClassifier(max_depth=20, random_state=42) # 9649

#clf = RandomForestClassifier(max_features='auto', random_state=42) # warning
#clf = RandomForestClassifier(max_features='sqrt', random_state=42) # 9649
#clf = RandomForestClassifier(max_features='log2', random_state=42) # 9649

#clf = RandomForestClassifier(min_samples_split=2, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_split=5, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_split=10, random_state=42) # 9649

#clf = RandomForestClassifier(min_samples_leaf=1, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_leaf=2, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_leaf=4, random_state=42) # 9649

clf = RandomForestClassifier(n_estimators=50, max_depth=20, max_features='sqrt', min_samples_split=5, min_samples_leaf=2, random_state=42)


clf.fit(X_train, y_train)
pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, pred)
print(f"Accuracy : {accuracy:.4f}")

Accuracy : 0.9649


In [3]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# 1. 데이터 로드
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']

# na_values = ? 는 ?로 되어있는 값들을 None값으로 처리한다는 의미
data = pd.read_csv(url, header=None, names=columns, na_values='?', skipinitialspace=True)

data.dropna(inplace=True)

Q1 = data['fnlwgt'].quantile(0.25)
Q3 = data['fnlwgt'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
capital_fnlwgt_outliers = data[(data['fnlwgt'] < lower_bound) | (data['fnlwgt'] > upper_bound)]
data = data.drop(capital_fnlwgt_outliers.index)

Q1 = data['capital-gain'].quantile(0.25)
Q3 = data['capital-gain'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
capital_gain_outliers = data[(data['capital-gain'] < lower_bound) | (data['capital-gain'] > upper_bound)]
capital_loss_outliers = data[(data['capital-loss'] < lower_bound) | (data['capital-loss'] > upper_bound)]
data = data.drop(capital_gain_outliers.index)
data = data.drop(capital_loss_outliers.index)

# 범주형 변수 인코딩
categorical_features = ['race', 'sex', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'income']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# 변수 선택 및 데이터 분리

X = data.drop('income_>50K', axis=1)
y = data['income_>50K']

# 파생변수1 : age_group
data['age_group'] = pd.cut(data['age'], bins=[0, 18, 30, 45, 60, 100], labels=['0-18', '19-30', '31-45', '46-60', '61+'])

# 파생변수2 : hours_group
data['hours_group'] = pd.cut(data['hours-per-week'], bins=[0, 20, 40, 60, 100], labels=['0-20', '21-40', '41-60', '61+'])

# 파생변수3 : capital
data['capital'] = data['capital-gain'] - data['capital-loss']

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[LightGBM] [Info] Number of positive: 4070, number of negative: 16245
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016665 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 643
[LightGBM] [Info] Number of data points in the train set: 20315, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200345 -> initscore=-1.384142
[LightGBM] [Info] Start training from score -1.384142
Best parameters for LightGBM: {'learning_rate': 0.2, 'n_estimators': 50, 'num_leaves': 31}


In [ ]:

########################################
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=3, n_jobs=-1, verbose=2)
grid_search_lr.fit(X_train, y_train)
print(f"Best parameters for Logistic Regression: {grid_search_lr.best_params_}")


########################################
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=3, n_jobs=-1, verbose=2)
grid_search_dt.fit(X_train, y_train)
print(f"Best parameters for Decision Tree: {grid_search_dt.best_params_}")


######################################
param_grid_gbm = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search_gbm = GridSearchCV(GradientBoostingClassifier(), param_grid_gbm, cv=3, n_jobs=-1, verbose=2)
grid_search_gbm.fit(X_train, y_train)
print(f"Best parameters for GBM: {grid_search_gbm.best_params_}")

######################################
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

grid_search_svm = GridSearchCV(SVC(), param_grid_svm, cv=3, n_jobs=-1, verbose=2)
grid_search_svm.fit(X_train, y_train)
print(f"Best parameters for SVM: {grid_search_svm.best_params_}")


################################
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=3, n_jobs=-1, verbose=2)
grid_search_knn.fit(X_train, y_train)
print(f"Best parameters for KNN: {grid_search_knn.best_params_}")

In [ ]:
################################
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search_xgb = GridSearchCV(xgb.XGBClassifier(), param_grid_xgb, cv=3, n_jobs=-1, verbose=2)
grid_search_xgb.fit(X_train, y_train)
print(f"Best parameters for XGBoost: {grid_search_xgb.best_params_}")

######################################
param_grid_lgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100]
}

grid_search_lgb = GridSearchCV(lgb.LGBMClassifier(), param_grid_lgb, cv=3, n_jobs=-1, verbose=2)
grid_search_lgb.fit(X_train, y_train)
print(f"Best parameters for LightGBM: {grid_search_lgb.best_params_}")

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. 데이터 로드
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']

# na_values = ? 는 ?로 되어있는 값들을 None값으로 처리한다는 의미
data = pd.read_csv(url, header=None, names=columns, na_values='?', skipinitialspace=True)

data.dropna(inplace=True)

Q1 = data['fnlwgt'].quantile(0.25)
Q3 = data['fnlwgt'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
capital_fnlwgt_outliers = data[(data['fnlwgt'] < lower_bound) | (data['fnlwgt'] > upper_bound)]
data = data.drop(capital_fnlwgt_outliers.index)

Q1 = data['capital-gain'].quantile(0.25)
Q3 = data['capital-gain'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
capital_gain_outliers = data[(data['capital-gain'] < lower_bound) | (data['capital-gain'] > upper_bound)]
capital_loss_outliers = data[(data['capital-loss'] < lower_bound) | (data['capital-loss'] > upper_bound)]
data = data.drop(capital_gain_outliers.index)
data = data.drop(capital_loss_outliers.index)

# 범주형 변수 인코딩
categorical_features = ['race', 'sex', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'income']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# 변수 선택 및 데이터 분리

X = data.drop('income_>50K', axis=1)
y = data['income_>50K']

# 파생변수1 : age_group
data['age_group'] = pd.cut(data['age'], bins=[0, 18, 30, 45, 60, 100], labels=['0-18', '19-30', '31-45', '46-60', '61+'])

# 파생변수2 : hours_group
data['hours_group'] = pd.cut(data['hours-per-week'], bins=[0, 20, 40, 60, 100], labels=['0-20', '21-40', '41-60', '61+'])

# 파생변수3 : capital
data['capital'] = data['capital-gain'] - data['capital-loss']

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# BestParameter
model = RandomForestClassifier(max_depth= 20, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 10, n_estimators= 200, random_state=10)
#{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# 정확도 평가
print(f"Matrix: {conf_matrix}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Report: {class_report}")
print(f"Accuracy: {accuracy_score}")

Matrix: [[3934  183]
 [ 538  424]]
Accuracy: 0.8580
Report:               precision    recall  f1-score   support

       False       0.88      0.96      0.92      4117
        True       0.70      0.44      0.54       962

    accuracy                           0.86      5079
   macro avg       0.79      0.70      0.73      5079
weighted avg       0.85      0.86      0.84      5079

Accuracy: <function accuracy_score at 0x7861e8187be0>
